In [1]:
!pip install -qU langchain_community pypdf
!pip install langchain-groq
!pip install sentence_transformers
!pip install faiss-gpu
!pip install tiktoken

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.4 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.2 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.9.4 requires shapely<2.1,>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
pointpats 2.5.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.1 requires shapely>=2.0.

In [33]:
!pip install youtube-transcript-api==0.6.2
!pip install pytube

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['GROQ_API_KEY'] = "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"

In [3]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq

# Logical Routing

In [5]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs",
                        "wikipedia", "arxiv", "pubmed",
                        "internal_docs", "product_manuals",
                        "weather_api", "maps_api","legal_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

In [6]:
# LLm with function call
llm = ChatGroq(temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

In [8]:
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [17]:
question = """
i want to know that what type of product you are selling?
"""

result = router.invoke({"question": question})
print(result)

datasource='product_manuals'


# Semantic Routing

In [20]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

In [21]:
# Two prompts
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [22]:
# Embed prompts
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)
prompt_templates = [physics_template, math_template]
prompt_embeddings = hf_embeddings.embed_documents(prompt_templates)

In [23]:
# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = hf_embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)

In [24]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatGroq()
    | StrOutputParser()
)

In [26]:
print(chain.invoke("what is "))

Using MATH
The father of mathematics is a title that is often given to the ancient Greek mathematician, Pythagoras. Pythagoras is known for his contributions to geometry, number theory, and mathematical philosophy. He is best known for the Pythagorean theorem, which states that in a right triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides.

However, it is important to note that mathematics as a field has a long and complex history, and there were many mathematicians who made significant contributions before Pythagoras. For example, the ancient Egyptians and Babylonians developed sophisticated mathematical systems for use in engineering, astronomy, and commerce. In ancient India, mathematicians made significant contributions to the development of arithmetic and geometry. And in ancient China, mathematicians developed methods for solving equations and calculating areas and volum

# Query structuring for metadata filters

In [41]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/live/CzeLannZJ-E?si=8tfn2C6wRhQsNk65",language="hi", add_video_info=True).load()

docs[0].metadata

{'source': 'CzeLannZJ-E',
 'title': 'Free AI Course | Lecture 38 | RAG Systems',
 'description': 'Unknown',
 'view_count': 4977,
 'thumbnail_url': 'https://i.ytimg.com/vi/CzeLannZJ-E/hq720.jpg?sqp=-oaymwEmCIAKENAF8quKqQMa8AEB-AH-CYAC0AWKAgwIABABGEwgSihlMA8=&rs=AOn4CLC896By8GN3qspgAbF7CaKmTrwW9g',
 'publish_date': '2024-09-03 00:00:00',
 'length': 3290,
 'author': 'Irfan Malik'}